In [1]:
import dask.bag as db
from dotenv import load_dotenv
from src.data.storage import get_S3_fs
from src.features.image_paths import (
    get_children,
    compute_img,
    CITIES_PATH,
    ZONES_SUBPATH,
    IMGS_SUBPATH,
    IMG_PATH_PATTERN,
)
from dask import delayed

load_dotenv()


True

In [2]:
S3 = get_S3_fs()
zone_paths = [
    zone
    for city in get_children(S3, "", CITIES_PATH)
    for zone in get_children(S3, city, ZONES_SUBPATH)
]
img_paths = [
    delayed(get_children)(S3, zone_path, IMGS_SUBPATH) for zone_path in zone_paths
]
img_data = [
    compute_img(img_paths_zone, IMG_PATH_PATTERN) for img_paths_zone in img_paths
]
ddf = db.from_delayed(img_data).to_dataframe()

In [3]:
ddf.head()


,imgPath,city,zone,store,menuRow,format
0,food-delivery-crawler/data/images/deliveroo.it...,bergamo,alzano-lombardo,pony-burger,0,jpeg
1,food-delivery-crawler/data/images/deliveroo.it...,bergamo,alzano-lombardo,pony-burger,1,jpeg
2,food-delivery-crawler/data/images/deliveroo.it...,bergamo,alzano-lombardo,pony-burger,10,jpeg
3,food-delivery-crawler/data/images/deliveroo.it...,bergamo,alzano-lombardo,pony-burger,11,jpeg
4,food-delivery-crawler/data/images/deliveroo.it...,bergamo,alzano-lombardo,pony-burger,12,jpeg
